# Football Data Collection
- I am analysing only the 2015 premier legue matches. In order to collect data related to 2015, premier league, I need to indentify the competition key for  premier league matches.
- Collect all competition list for 2015 read json file and identify competion ID for Premier League
- Identify premier league id for year 2015 
- Hit second api to fetch match data for that id and store the JSON files in local directory

In [33]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [1]:
import requests
import os
import re
import csv
import pandas as pd
import json
import http.client
import time
import glob
key_pl =  os.getenv("key_pl")
#requests.get("http://api.football-data.org/v1/competitions/")
#to check 200 response!! But actually what you need to do this below

In [2]:
path_val = 'Football/competition_list/'

In [3]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        try:
            os.makedirs(file_path,exist_ok=True)
            
        except OSError as exception:
            if exception.errno != errno.EEXIST:
                raise

In [4]:

ind_article_list=[]
season_yr='2015'
connection = http.client.HTTPConnection('api.football-data.org')
headers = { 'X-Auth-Token': key_pl, 'X-Response-Control': 'minified' }
connection.request('GET', 'http://api.football-data.org/v1/competitions/?season='+season_yr, None, headers )
competition_list=json.loads(connection.getresponse().read().decode())
#competition_list=requests.get("http://api.football-data.org/v1/competitions/?season="+season_yr).json()
#competition_list

In [6]:
#competition_list

In [28]:
#Store Data in local file
file_name =path_val+ 'competition_list.json'
os.makedirs(os.path.dirname(file_name), exist_ok=True)
with open(file_name, 'w', encoding='utf-8') as f:
    json.dump(competition_list, f)
    f.close()

In [29]:
#Read competitoinList to idntify ID
directory='Football/competition_list'
for root, dirs, files in os.walk(directory):
     for file in files:
        with open(os.path.join(root, file), "r") as f:
            data = json.load(f)

In [30]:
#data[0]

In [31]:
pl_data=[]
pl_id=""
for l in data:
    if 'caption' in l and 'id' in l and 'league' in l:
        if l['league'] == 'PL':
            #print(data[l])
            pl_data.append(l)
            pl_id= l['id']
print("We have found that 2015 premier league's ID is "+str(pl_id))
pl_data  

We have found that 2015 premier league's ID is 398


[{'caption': 'Premier League 2015/16',
  'currentMatchday': 38,
  'id': 398,
  'lastUpdated': '2016-05-19T15:12:55Z',
  'league': 'PL',
  'numberOfGames': 380,
  'numberOfMatchdays': 38,
  'numberOfTeams': 20,
  'year': '2015'}]

In [43]:
match_list=[]
#for i in range(1,38):
#connection = http.client.HTTPConnection('api.football-data.org')
#headers = { 'X-Auth-Token': key_pl, 'X-Response-Control': 'minified' }
#connection.request('GET', 'http://api.football-data.org/v1/competitions/'+str(pl_id)+'/fixtures?', None, headers )
#match_list=json.loads(connection.getresponse().read().decode())

match_list=requests.get("http://api.football-data.org/v1/competitions/"+str(pl_id)+"/fixtures?").json()


In [45]:
match_list


{'error': 'You reached your request limit. Get your free API token to use the API extensively.'}

In [47]:
path_val_match = 'Football/Premier_League_2015_Data/'
match_list_index = []
for i in range(0,len(match_list['fixtures'])):
    match_list_index.append(match_list['fixtures'][i])
    #print(match_list['fixtures'][i])
    #break
    #print('step1')
    #for match in match_list['fixtures'][i]:
     #   match_list_index.append(match)
#match_list_index[0]
for match in match_list_index:
    awayTeamName = match['awayTeamName']
    homeTeamName = match['homeTeamName']
    matchday = match['matchday']
    if awayTeamName and homeTeamName and matchday:
        file_name_match =path_val_match+homeTeamName+'/'+homeTeamName+'vs'+ awayTeamName+'on Day'+str(matchday)
        os.makedirs(os.path.dirname(file_name_match), exist_ok=True)
        with open(file_name_match, 'w', encoding='utf-8') as f:
            json.dump(match, f)
            f.close()

        

KeyError: 'fixtures'

In [86]:
team_fix_dic={}
directory="Football/Premier_League_2015_Data"
fixture_list=[]
#for name in glob.glob("Football/Premier_League_2015_Data/*.*"):
for root, dirs, files in os.walk(directory):
    for file in files:
        with open(os.path.join(root, file), "r") as f:
            data2 = json.load(f)
    
            awayTeamName = data2['awayTeamName']
            homeTeamName = data2['homeTeamName']
            team1=awayTeamName+','+homeTeamName
            team2=homeTeamName+','+awayTeamName
            #print(team1)
            fixture =data2['_links']['self']['href'][data2['_links']['self']['href'].find('/fixtures/')+10:]
            if len(team_fix_dic) >0:
                if team1 not in team_fix_dic and team2 not in team_fix_dic :
                    #print("hi")
                    team_fix_dic[team1] = fixture
            else:
                team_fix_dic[team1] = fixture
        #break
#print(team_fix_dic)        

In [88]:
fixture_list=[]
for k,v in team_fix_dic.items():
    l=[k,v]
    fixture_list.append(l)
field_name=["Team","Fixture"]
with open("Football/match_fix_unique.csv", 'w',newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(field_name)
        writer.writerows(fixture_list) 

In [7]:
path_val_fix="Football/Head2Head/"
directory_fix_id= "Football/match_fix_unique.csv"
df_fixtureid=pd.read_csv(directory_fix_id,sep=',')
fixture_list=df_fixtureid.values.tolist()
for i in range(0,len(fixture_list)):
    v=fixture_list[i][1]
    k=fixture_list[i][0]
    connection = http.client.HTTPConnection('api.football-data.org')
    headers = { 'X-Auth-Token': key_pl, 'X-Response-Control': 'minified' }
    connection.request('GET', 'http://api.football-data.org/v1/fixtures/'+str(v)+'?head2head=50', None, headers )
    head2head_list=json.loads(connection.getresponse().read().decode())
    teams=k.split(',')
    #print(teams)
    file_name_match = 'Football/Head2Head/Head2HeadByFixture/'+teams[0]+'vs'+ teams[1]
    #os.makedirs(os.path.dirname(file_name_match))
    with open(file_name_match, 'w') as f:
        json.dump(head2head_list, f)
        f.close()
    #print(head2head_list)
    time.sleep(5)

KeyboardInterrupt: 